In [29]:
# Execute in Windows Prompt or Terminal (MAC)
# !gcloud auth application-default login

In [30]:
# !gcloud config set project hca-hin-dev-cur-parallon

In [31]:
from google.cloud import bigquery
import google.auth

In [32]:
credentials, project_id = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
print(project_id)
bq_client = bigquery.Client(credentials=credentials, project=project_id)

C:\Users\KHU9683\AppData\Roaming\Python\Python311\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


hca-hin-dev-cur-parallon


In [33]:
import os
os.environ["HTTP_PROXY"] = "proxy.nas.medcity.net:80"
os.environ["HTTPS_PROXY"] = "proxy.nas.medcity.net:80"

In [34]:
from datetime import datetime, timedelta, time
from jinja2 import Template

In [35]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [36]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = file_data
        file.write(file_string)

In [37]:
template_folder = "InputFiles\Template"
template_name = "pagl_template"
jinja_extension = ".j2"
html_extension = ".html"
output_path_folder = f"OutputFiles\HTML"
output_file_name = "PAGL_Subject"

In [38]:
run_time = datetime.now().time()
check_time = time(10,00)
print("Run Time : {}".format(run_time))
print("Check Time : {}".format(check_time))
if run_time > check_time:
    day_ind = 1
else:
    day_ind = 2

Run Time : 14:05:52.773882
Check Time : 10:00:00


In [39]:
target_query = "SELECT Stack, COUNT(*) as Rec_Count FROM edwra_staging.brbglx WHERE CAST(entered_ts AS DATE) = current_date('US/Central') - {} GROUP BY Stack ORDER BY Stack ASC".format(day_ind)
target_rs = bq_client.query(target_query, project=project_id, location='US').result()

In [40]:
rec_list = []
for target_row in target_rs:
    rec_dict_item = {"Stack":target_row.Stack, "Rec_Count":target_row.Rec_Count}
    rec_list.append(rec_dict_item)
print(rec_list)

[{'Stack': 'A', 'Rec_Count': 140123}, {'Stack': 'F', 'Rec_Count': 447330}, {'Stack': 'G', 'Rec_Count': 304631}, {'Stack': 'H', 'Rec_Count': 233801}, {'Stack': 'I', 'Rec_Count': 106480}, {'Stack': 'J', 'Rec_Count': 150184}, {'Stack': 'K', 'Rec_Count': 243663}, {'Stack': 'T', 'Rec_Count': 25283}]


In [41]:
html_dict = {
    "notify_name": "Merwin",
    "rec_list":rec_list
}

In [42]:
#get the DDLs for the tables from teradata 
def gen_script_using_template():
    dt1 = datetime.now()

    try:
        if not os.path.exists(output_path_folder):
            os.makedirs(output_path_folder)

        template_file_path = f"{template_folder}\{template_name}{jinja_extension}"
        content_template = read_contents_file(template_file_path)
        template = Template(content_template)
        output_filename_path = f"{output_path_folder}\{output_file_name}{html_extension}"
        write_file_local(output_filename_path, template.render(html_dict))

    except Exception as e1:
        print(e1)
        print("File Name : {}".format(template_file_name))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [43]:
print("Begin of Processing")

gen_script_using_template()

print("End of Processing")

Begin of Processing
0:00:00.003997
End of Processing
